<a href="https://colab.research.google.com/github/Dazhou2000/Msc-Project/blob/master/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
import optuna
import json
import os

def load_data(patient_id, window_size):
    data_path = f'/content/drive/My Drive/Msc Project/features_and_labels_detection/P{patient_id} features and seizure labels with window_{window_size}.csv'
    data = pd.read_csv(data_path)
    data = data.dropna()
    importance = pd.read_csv(f'/content/drive/My Drive/Msc Project/importance_feature/P{patient_id}MDI_importance with window_{window_size}.csv')
    top_features_list = importance.sort_values(by='Importance', ascending=False)['Unnamed: 0'].tolist()
    return data, top_features_list

def save_best_params(patient_id, window_size, best_params):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/best_params_xgb'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/best_params_xgb')

    params_file = f'/content/drive/My Drive/Msc Project/best_params_xgb/params_patient_{patient_id}_window_{window_size}.json'
    with open(params_file, 'w') as file:
        json.dump(best_params, file)

def load_best_params(patient_id, window_size):
    params_file = f'/content/drive/My Drive/Msc Project/best_params_xgb/params_patient_{patient_id}_window_{window_size}.json'
    if os.path.exists(params_file):
        with open(params_file, 'r') as file:
            best_params = json.load(file)
        return best_params
    else:
        return None

def save_best_scores(patient_id, window_size, best_scores):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/best_scores_xgb'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/best_scores_xgb')
    scores_file = f'/content/drive/My Drive/Msc Project/best_scores_xgb/best_scores_more.csv'
    row_name = f'{patient_id}_window_{window_size}'
    scores_df = pd.DataFrame(best_scores, index=[row_name])
    if os.path.exists(scores_file):
        existing_df = pd.read_csv(scores_file, index_col=0)
        if row_name in existing_df.index:
            existing_df.loc[row_name] = scores_df.loc[row_name]
            existing_df.to_csv(scores_file)
        else:
            updated_df = pd.concat([existing_df, scores_df])
            updated_df.to_csv(scores_file)
    else:
        scores_df.to_csv(scores_file)

# Example usage:Patient id:'1_','3_','4_','10','13','19','23','27','29','30','31'. Window_size: 24,72,144,288
patient_id = '31'
window_size = 288

data, top_features_list = load_data(patient_id, window_size)

selected_features = pd.read_csv(f'/content/drive/My Drive/Msc Project/select_feature/P{patient_id}_selected_features_with_window_{window_size}.csv')
number_features = selected_features.loc[selected_features['AUC'].idxmax()]['Num Features']
print(number_features)

X = data.drop(columns=['label', 'timestamp', 'seizure_event', 'Unnamed: 0'])
X = data[top_features_list[:int(number_features)]]
y = data['label']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)
smote = SMOTE(random_state=42, n_jobs=-1)
X_train, y_train = smote.fit_resample(X_train, y_train)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

def objective(trial):
    params = {
        'verbosity': 1,
        'lambda': trial.suggest_float('lambda', 1.0, 2.0,log=True),
        'alpha': trial.suggest_float('alpha', 1e-5, 100.0,log=True),
        'max_depth':trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_float('eta', 1e-5, 1.0,log=True),
        'gamma': trial.suggest_float('gamma', 0.1, 0.2,log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'grow_policy':trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }


    xgb_model = XGBClassifier(**params, use_label_encoder=False, eval_metric='auc', tree_method='hist', random_state=42,device='gpu',booster='gbtree',objective='binary:logistic')
    xgb_model.fit(X_train, y_train)
    y_pred_val = xgb_model.predict(X_val)
    y_pred_val_scores = xgb_model.predict_proba(X_val)[:,1]
    auc_val = roc_auc_score(y_val, y_pred_val_scores)
    return auc_val

study = optuna.create_study(direction='maximize')

best_params = load_best_params(patient_id, window_size)
if best_params:
    study.enqueue_trial(best_params)

study.optimize(objective, n_trials=1)

best_params = study.best_params
print("Best parameters found by Optuna:", best_params)

save_best_params(patient_id, window_size, best_params)

xgb_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='auc', tree_method='hist', random_state=42,device='gpu',booster='gbtree',objective='binary:logistic')
xgb_model.fit(X_train, y_train)

y_pred_train = xgb_model.predict(X_train)
y_pred_val = xgb_model.predict(X_val)
y_pred_test = xgb_model.predict(X_test)

# Calculate roc_auc_score
y_pred_train_scores = xgb_model.predict_proba(X_train)[:,1]
auc_train = roc_auc_score(y_train, y_pred_train)
y_pred_val_scores = xgb_model.predict_proba(X_val)[:,1]
auc_val = roc_auc_score(y_val, y_pred_val_scores)
y_pred_test_scores = xgb_model.predict_proba(X_test)[:,1]
auc_test = roc_auc_score(y_test, y_pred_test_scores)

print("Train classification report:")
print(classification_report(y_train, y_pred_train))

print("Validation classification report:")
print(classification_report(y_val, y_pred_val))

print("Test classification report:")
print(classification_report(y_test, y_pred_test))

print("Train accuracy:", accuracy_score(y_train, y_pred_train))
print("Validation accuracy:", accuracy_score(y_val, y_pred_val))
print("Test accuracy:", accuracy_score(y_test, y_pred_test))

print("Val auc:", auc_val)
print("Test auc:", auc_test)
print("Train auc:", auc_train)

best_scores = {
    "sensitivity": classification_report(y_test, y_pred_test, output_dict=True)['1']['recall'],
    "auc": auc_test,
    "specificity": classification_report(y_test, y_pred_test, output_dict=True)['0']['recall'],
    "accuracy": accuracy_score(y_test, y_pred_test)
}

save_best_scores(patient_id, window_size, best_scores)


19.0


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-17 16:05:11,249] A new study created in memory with name: no-name-495b5ffc-83be-44c1-bc1a-66da5e28e32c
[I 2024-07-17 16:05:13,661] Trial 0 finished with value: 0.7643178015441149 and parameters: {'lambda': 1.5858877414042443, 'alpha': 0.007376108351406913, 'max_depth': 9, 'eta': 0.014376023577920038, 'gamma': 0.16250083904882537, 'subsample': 0.9423017241583421, 'colsample_bytree': 0.7715534237052748, 'grow_policy': 'lossguide', 'min_child_weight': 9}. Best is trial 0 with value: 0.7643178015441149.


Best parameters found by Optuna: {'lambda': 1.5858877414042443, 'alpha': 0.007376108351406913, 'max_depth': 9, 'eta': 0.014376023577920038, 'gamma': 0.16250083904882537, 'subsample': 0.9423017241583421, 'colsample_bytree': 0.7715534237052748, 'grow_policy': 'lossguide', 'min_child_weight': 9}
Train classification report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     81314
           1       0.97      0.96      0.96     81314

    accuracy                           0.96    162628
   macro avg       0.96      0.96      0.96    162628
weighted avg       0.96      0.96      0.96    162628

Validation classification report:
              precision    recall  f1-score   support

           0       0.76      0.89      0.82      9623
           1       0.64      0.40      0.49      4602

    accuracy                           0.73     14225
   macro avg       0.70      0.64      0.65     14225
weighted avg       0.72      0.73      0.71 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
import optuna
import json
import os
import shutil
import random
from xgboost import XGBClassifier

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

def load_data(patient_id, window_size, forecasting_length):
    data_path = f'/content/drive/My Drive/Msc Project/features_and_labels_detection/P{patient_id} features and seizure labels with window_{window_size}.csv'
    data = pd.read_csv(data_path)
    data = data.dropna()
    importance = pd.read_csv(f'/content/drive/My Drive/Msc Project/Forecasting/importance_feature/P{patient_id}MDI_importance with window_{window_size} with forecasting_{forecasting_length}.csv')
    top_features_list = importance.sort_values(by='Importance', ascending=False)['Unnamed: 0'].tolist()
    return data, top_features_list

def save_best_params(patient_id, window_size, forecasting_length, best_params):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/Forecasting/best_params_xgb'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/Forecasting/best_params_xgb')

    params_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_params_xgb/params_patient_{patient_id}_window_{window_size}_forecasting_{forecasting_length}.json'
    with open(params_file, 'w') as file:
        json.dump(best_params, file)

def load_best_params(patient_id, window_size, forecasting_length):
    params_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_params_xgb/params_patient_{patient_id}_window_{window_size}_forecasting_{forecasting_length}.json'
    if os.path.exists(params_file):
        with open(params_file, 'r') as file:
            best_params = json.load(file)
        return best_params
    else:
        return None

def save_best_scores(patient_id, window_size, forecasting_length, best_scores):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_xgb'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_xgb')
    scores_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_xgb/best_scores.csv'
    row_name = f'{patient_id}_window_{window_size}_forecasting_{forecasting_length}'
    scores_df = pd.DataFrame(best_scores, index=[row_name])
    if os.path.exists(scores_file):
        existing_df = pd.read_csv(scores_file, index_col=0)
        if row_name in existing_df.index:
            existing_df.loc[row_name] = scores_df.loc[row_name]
            existing_df.to_csv(scores_file)
        else:
            updated_df = pd.concat([existing_df, scores_df])
            updated_df.to_csv(scores_file)
    else:
        scores_df.to_csv(scores_file)

def process_patient_window(patient_id, window_size, forecasting_length):
    data, top_features_list = load_data(patient_id, window_size, forecasting_length)

    selected_features = pd.read_csv(f'/content/drive/My Drive/Msc Project/Forecasting/select_feature/P{patient_id}_selected_features_with_window_{window_size}_with_forecasting_{forecasting_length}.csv')
    number_features = selected_features.loc[selected_features['AUC'].idxmax()]['Num Features']
    print(f'Number of features for patient {patient_id} with window {window_size}: {number_features}')

    X = data.drop(columns=['label', 'timestamp', 'seizure_event', 'Unnamed: 0'])
    X = X[top_features_list[:int(number_features)]]
    y = data['label']
    y = y.shift(-forecasting_length)

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)
    smote = SMOTE(random_state=42, n_jobs=-1)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)
    print(len(X_test))
    print(len(y_test))
    X_test = X_test.iloc[:-forecasting_length]
    y_test = y_test.iloc[:-forecasting_length]
    print(len(X_test))
    print(len(y_test))
    def objective(trial):
        params = {
            'verbosity': 1,
            'lambda': trial.suggest_float('lambda', 1.0, 2.0, log=True),
            'alpha': trial.suggest_float('alpha', 1e-5, 100.0, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'eta': trial.suggest_float('eta', 1e-5, 1.0, log=True),
            'gamma': trial.suggest_float('gamma', 0.1, 0.2, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        }

        xgb_model = XGBClassifier(**params, use_label_encoder=False, eval_metric='auc', tree_method='hist', random_state=42, device='gpu', booster='gbtree', objective='binary:logistic')
        xgb_model.fit(X_train, y_train)
        y_pred_val = xgb_model.predict(X_val)
        y_pred_val_scores = xgb_model.predict_proba(X_val)[:, 1]
        auc_val = roc_auc_score(y_val, y_pred_val_scores)
        return auc_val

    class EarlyStoppingCallback:
        def __init__(self, patience: int):
            self.patience = patience
            self.best_value = None
            self.no_improvement_count = 0

        def __call__(self, study, trial):
            if self.best_value is None or study.best_value > self.best_value:
                self.best_value = study.best_value
                self.no_improvement_count = 0
            else:
                self.no_improvement_count += 1
            if self.no_improvement_count >= self.patience:
                study.stop()

    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))

    best_params = load_best_params(patient_id, window_size, forecasting_length)
    if best_params:
        study.enqueue_trial(best_params)

    early_stopping = EarlyStoppingCallback(patience=50)
    study.optimize(objective, n_trials=200, callbacks=[early_stopping])

    best_params = study.best_params
    print("Best parameters found by Optuna:", best_params)

    save_best_params(patient_id, window_size, forecasting_length, best_params)

    xgb_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='auc', tree_method='hist', random_state=42, device='gpu', booster='gbtree', objective='binary:logistic')
    xgb_model.fit(X_train, y_train)

    y_pred_train = xgb_model.predict(X_train)
    y_pred_val = xgb_model.predict(X_val)
    y_pred_test = xgb_model.predict(X_test)

    y_pred_train_scores = xgb_model.predict_proba(X_train)[:, 1]
    auc_train = roc_auc_score(y_train, y_pred_train_scores)
    y_pred_val_scores = xgb_model.predict_proba(X_val)[:, 1]
    auc_val = roc_auc_score(y_val, y_pred_val_scores)
    y_pred_test_scores = xgb_model.predict_proba(X_test)[:, 1]
    auc_test = roc_auc_score(y_test, y_pred_test_scores)

    print(f"Train classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_train, y_pred_train))

    print(f"Validation classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_val, y_pred_val))

    print(f"Test classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_test, y_pred_test))

    print(f"Train accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_train, y_pred_train)}")
    print(f"Validation accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_val, y_pred_val)}")
    print(f"Test accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_test, y_pred_test)}")

    print(f"Train AUC for patient {patient_id} with window {window_size}: {auc_train}")
    print(f"Validation AUC for patient {patient_id} with window {window_size}: {auc_val}")
    print(f"Test AUC for patient {patient_id} with window {window_size}: {auc_test}")

    best_scores = {
        "auc": auc_test,
        "sensitivity": classification_report(y_test, y_pred_test, output_dict=True)['1.0']['recall'],
        "specificity": classification_report(y_test, y_pred_test, output_dict=True)['0.0']['recall'],
        "accuracy": accuracy_score(y_test, y_pred_test)
    }

    save_best_scores(patient_id, window_size, forecasting_length, best_scores)

patient_ids = ['3_','4_','13','23','29','31']
window_sizes = [288]
forecasting_lengths = [24, 120]

for patient_id in patient_ids:
    for window_size in window_sizes:
        for forecasting_length in forecasting_lengths:
            print(f"Processing patient {patient_id} with window size {window_size} with forecasting length {forecasting_length}")
            process_patient_window(patient_id, window_size, forecasting_length)


Processing patient 3_ with window size 288 with forecasting length 24
Number of features for patient 3_ with window 288: 7.0
12462
12462


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-20 20:38:43,926] A new study created in memory with name: no-name-df496bb2-6009-44c3-b4af-d03bc4c75673


12438
12438


[I 2024-07-20 20:38:45,266] Trial 0 finished with value: 0.5028203318803841 and parameters: {'lambda': 1.296426233049547, 'alpha': 45.18560951024108, 'max_depth': 8, 'eta': 0.009846738873614562, 'gamma': 0.11142080469295271, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'grow_policy': 'depthwise', 'min_child_weight': 8}. Best is trial 0 with value: 0.5028203318803841.
[I 2024-07-20 20:38:46,336] Trial 1 finished with value: 0.5227575818437438 and parameters: {'lambda': 1.0143703591411894, 'alpha': 61.56997328235202, 'max_depth': 9, 'eta': 0.00011526449540315612, 'gamma': 0.11343178571896925, 'subsample': 0.5917022549267169, 'colsample_bytree': 0.6521211214797689, 'grow_policy': 'depthwise', 'min_child_weight': 3}. Best is trial 1 with value: 0.5227575818437438.
[I 2024-07-20 20:38:46,859] Trial 2 finished with value: 0.5024461657882023 and parameters: {'lambda': 1.5282206868683035, 'alpha': 9.472334467618546e-05, 'max_depth': 5, 'eta': 0.0006789053271698484, 

Best parameters found by Optuna: {'lambda': 1.8264210482776209, 'alpha': 0.025154562154085562, 'max_depth': 8, 'eta': 0.048024786180828824, 'gamma': 0.14183878446298256, 'subsample': 0.9960016289401145, 'colsample_bytree': 0.5017161308413209, 'grow_policy': 'depthwise', 'min_child_weight': 9}
Train classification report for patient 3_ with window 288:
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98     83211
         1.0       0.98      0.99      0.98     83211

    accuracy                           0.98    166422
   macro avg       0.98      0.98      0.98    166422
weighted avg       0.98      0.98      0.98    166422

Validation classification report for patient 3_ with window 288:
              precision    recall  f1-score   support

         0.0       0.78      0.97      0.86      9575
         1.0       0.42      0.07      0.12      2886

    accuracy                           0.76     12461
   macro avg       0.60      0.52   

...

Best parameters found by Optuna: {'lambda': 1.000664798783858, 'alpha': 0.0010040919006327135, 'max_depth': 10, 'eta': 0.0034806816803432997, 'gamma': 0.17014573977328223, 'subsample': 0.8043372503333879, 'colsample_bytree': 0.563310358073504, 'grow_policy': 'depthwise', 'min_child_weight': 3}
Train classification report for patient 31 with window 288:
              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96     81314
         1.0       0.97      0.95      0.96     81314

    accuracy                           0.96    162628
   macro avg       0.96      0.96      0.96    162628
weighted avg       0.96      0.96      0.96    162628

Validation classification report for patient 31 with window 288:
              precision    recall  f1-score   support

         0.0       0.76      0.93      0.84      9623
         1.0       0.74      0.39      0.51      4602

    accuracy                           0.76     14225
   macro avg       0.75      0.66  

/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-20 20:55:42,458] A new study created in memory with name: no-name-f15f48dc-c2aa-467c-9bc7-98d89ea3dc05


14225
14225
14105
14105


[I 2024-07-20 20:55:43,451] Trial 0 finished with value: 0.6528843285858921 and parameters: {'lambda': 1.296426233049547, 'alpha': 45.18560951024108, 'max_depth': 8, 'eta': 0.009846738873614562, 'gamma': 0.11142080469295271, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'grow_policy': 'depthwise', 'min_child_weight': 8}. Best is trial 0 with value: 0.6528843285858921.
[I 2024-07-20 20:55:44,484] Trial 1 finished with value: 0.6663586040537423 and parameters: {'lambda': 1.0143703591411894, 'alpha': 61.56997328235202, 'max_depth': 9, 'eta': 0.00011526449540315612, 'gamma': 0.11343178571896925, 'subsample': 0.5917022549267169, 'colsample_bytree': 0.6521211214797689, 'grow_policy': 'depthwise', 'min_child_weight': 3}. Best is trial 1 with value: 0.6663586040537423.
[I 2024-07-20 20:55:45,475] Trial 2 finished with value: 0.6474350630443086 and parameters: {'lambda': 1.5282206868683035, 'alpha': 9.472334467618546e-05, 'max_depth': 5, 'eta': 0.0006789053271698484, 

Best parameters found by Optuna: {'lambda': 1.4832960581947787, 'alpha': 10.046912074111086, 'max_depth': 4, 'eta': 0.03170965272282716, 'gamma': 0.10460278325594857, 'subsample': 0.544436214506524, 'colsample_bytree': 0.5049939450509008, 'grow_policy': 'depthwise', 'min_child_weight': 9}
Train classification report for patient 31 with window 288:
              precision    recall  f1-score   support

         0.0       0.73      0.84      0.78     81314
         1.0       0.81      0.69      0.75     81314

    accuracy                           0.77    162628
   macro avg       0.77      0.77      0.76    162628
weighted avg       0.77      0.77      0.76    162628

Validation classification report for patient 31 with window 288:
              precision    recall  f1-score   support

         0.0       0.75      0.80      0.77      9571
         1.0       0.52      0.45      0.48      4654

    accuracy                           0.68     14225
   macro avg       0.63      0.62      0